In [78]:
import numpy as np
import requests
import pandas as pd
import sys

In [79]:
#Useful functions

#df_col_astype_int: Intakes the dataframe and corrects the types of the values in each column 
def df_col_astype_int(df,col_names,col_names_float):
    '''Inputs:
            df: A pandas Dataframe.
            col_names: Names of the columns in df.
            col_names_float: Names of the columns that instead of having to be integers need to be floats.
            
        Outputs:
            df but with the correct columns changed to integers and floats'''
    
    for name in col_names:
        if name not in col_names_float:
            df[name] = df[name].astype(int)
        elif name in col_names_float:
            df[name] = df[name].astype(float)
        else:
            print("Error: Unaccounted for name in col_names list. Revise these lists")
            sys.exit()
    return df


In [5]:
#Load all the player history data from Fantasy Premier League (FPL)

fpl_data = requests.get('https://fantasy.premierleague.com/drf/bootstrap-static').json()

for i, player in enumerate(fpl_data['elements']):
    fpl_data['elements'][i]['history'] = requests.get('https://fantasy.premierleague.com/drf/element-summary/' + str(player['id'])).json()

In [80]:
#Points threshold
threshold_points = 8

#Position Booleans
goalkeeper = False
defender = False
midfielder = False
striker = True


if goalkeeper == True:
    position = 1
elif defender == True:
    position = 2
elif midfielder == True:
    position = 3
elif striker == True:
    position = 4
else:
    print("Error: Invalid position value ")
    sys.exit()
    
#Feature names are specific to whather an individual player is outfield or in goal
#Featue details should be obvious by name but more info can be found at https://fantasy.premierleague.com/help/
#Feature selection under decision after considering correlations between them in the future.
if goalkeeper == False:
    column_names = ['element','GW','minutes','goals_scored','assists','clean_sheets','goals_conceded','own_goals',
                    'penalties_missed','yellow_cards','red_cards','bonus','bps','influence','creativity','threat',
                    'ict_index','open_play_crosses','big_chances_created','clearances_blocks_interception','recoveries',
                    'key_passes','tackles','attempted_passes','completed_passes','big_chances_missed',
                    'errors_leading_to_goal','errors_leading_to_goal_attempt','tackled','offside','target_missed',
                    'fouls','dribbles','total_points']
    df = pd.DataFrame(columns=column_names)
elif goalkeeper == True:
    column_names = ['element','GW','minutes','clean_sheets','goals_conceded','own_goals','penalties_missed',
                    'yellow_cards','red_cards','bonus','bps','influence','creativity','threat','ict_index',
                    'big_chances_created','clearances_blocks_interception','recoveries','key_passes','tackles',
                    'attempted_passes','completed_passes','big_chances_missed','errors_leading_to_goal','errors_leading_to_goal_attempt',
                    'tackled','penalties_saved','saves','penalties_conceded','total_points']
    df = pd.DataFrame(columns=column_names)

print('Position tuning complete')

Position tuning complete


In [81]:
#Creating the complete player history dataframe for players of a particular position

for i, player in enumerate(fpl_data['elements']):
    if fpl_data['elements'][i]['element_type'] == position: 
        print('ID: ',fpl_data['elements'][i]['id'], " Name: ",fpl_data['elements'][i]['web_name'])
        for m in range(0,len(fpl_data['elements'][i]['history']['history'])):
            if goalkeeper == False:
                                
                element = int(fpl_data['elements'][i]['history']['history'][m]['element'])
                GW = int(fpl_data['elements'][i]['history']['history'][m]['round'])
                minutes = int(fpl_data['elements'][i]['history']['history'][m]['minutes'])
                goals_scored = int(fpl_data['elements'][i]['history']['history'][m]['goals_scored'])
                assists = int(fpl_data['elements'][i]['history']['history'][m]['assists'])
                clean_sheets = int(fpl_data['elements'][i]['history']['history'][m]['clean_sheets'])
                goals_conceded = int(fpl_data['elements'][i]['history']['history'][m]['goals_conceded'])
                own_goals = int(fpl_data['elements'][i]['history']['history'][m]['own_goals'])
                penalties_missed = int(fpl_data['elements'][i]['history']['history'][m]['penalties_missed'])
                yellow_cards = int(fpl_data['elements'][i]['history']['history'][m]['yellow_cards'])
                red_cards = int(fpl_data['elements'][i]['history']['history'][m]['red_cards'])
                bonus = int(fpl_data['elements'][i]['history']['history'][m]['bonus'])
                bps = int(fpl_data['elements'][i]['history']['history'][m]['bps'])
                influence = float(fpl_data['elements'][i]['history']['history'][m]['influence'])
                creativity = float(fpl_data['elements'][i]['history']['history'][m]['creativity'])
                threat = float(fpl_data['elements'][i]['history']['history'][m]['threat'])
                ict_index = float(fpl_data['elements'][i]['history']['history'][m]['ict_index'])
                open_play_crosses = int(fpl_data['elements'][i]['history']['history'][m]['open_play_crosses'])
                big_chances_created = int(fpl_data['elements'][i]['history']['history'][m]['big_chances_created'])
                clearances_blocks_interception = int(fpl_data['elements'][i]['history']['history'][m]['clearances_blocks_interceptions'])
                recoveries = int(fpl_data['elements'][i]['history']['history'][m]['recoveries'])
                key_passes = int(fpl_data['elements'][i]['history']['history'][m]['key_passes'])
                tackles = int(fpl_data['elements'][i]['history']['history'][m]['tackles'])
                attempted_passes = int(fpl_data['elements'][i]['history']['history'][m]['attempted_passes'])
                completed_passes = int(fpl_data['elements'][i]['history']['history'][m]['completed_passes'])
                big_chances_missed = int(fpl_data['elements'][i]['history']['history'][m]['big_chances_missed'])
                errors_leading_to_goal = int(fpl_data['elements'][i]['history']['history'][m]['errors_leading_to_goal'])
                errors_leading_to_goal_attempt = int(fpl_data['elements'][i]['history']['history'][m]['errors_leading_to_goal_attempt'])
                tackled = int(fpl_data['elements'][i]['history']['history'][m]['tackled'])
                offside = int(fpl_data['elements'][i]['history']['history'][m]['offside'])
                target_missed = int(fpl_data['elements'][i]['history']['history'][m]['target_missed'])
                fouls = int(fpl_data['elements'][i]['history']['history'][m]['fouls'])
                dribbles = int(fpl_data['elements'][i]['history']['history'][m]['dribbles'])

                total_points = int(fpl_data['elements'][i]['history']['history'][m]['total_points'])
                
                series = pd.Series([element,GW,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,
                    penalties_missed,yellow_cards,red_cards,bonus,bps,influence,creativity,threat,
                    ict_index,open_play_crosses,big_chances_created,clearances_blocks_interception,recoveries,
                    key_passes,tackles,attempted_passes,completed_passes,big_chances_missed,
                    errors_leading_to_goal,errors_leading_to_goal_attempt,tackled,offside,target_missed,
                    fouls,dribbles,total_points],index=column_names)

            elif goalkeeper == True:

                element = int(fpl_data['elements'][i]['history']['history'][m]['element'])
                GW = int(fpl_data['elements'][i]['history']['history'][m]['round'])
                minutes = int(fpl_data['elements'][i]['history']['history'][m]['minutes'])
                clean_sheets = int(fpl_data['elements'][i]['history']['history'][m]['clean_sheets'])
                goals_conceded = int(fpl_data['elements'][i]['history']['history'][m]['goals_conceded'])
                own_goals = int(fpl_data['elements'][i]['history']['history'][m]['own_goals'])
                penalties_missed = int(fpl_data['elements'][i]['history']['history'][m]['penalties_missed'])
                yellow_cards = int(fpl_data['elements'][i]['history']['history'][m]['yellow_cards'])
                red_cards = int(fpl_data['elements'][i]['history']['history'][m]['red_cards'])
                bonus = int(fpl_data['elements'][i]['history']['history'][m]['bonus'])
                bps = int(fpl_data['elements'][i]['history']['history'][m]['bps'])
                influence = float(fpl_data['elements'][i]['history']['history'][m]['influence'])
                creativity = float(fpl_data['elements'][i]['history']['history'][m]['creativity'])
                threat = float(fpl_data['elements'][i]['history']['history'][m]['threat'])
                ict_index = float(fpl_data['elements'][i]['history']['history'][m]['ict_index'])
                big_chances_created = int(fpl_data['elements'][i]['history']['history'][m]['big_chances_created'])
                clearances_blocks_interceptions = int(fpl_data['elements'][i]['history']['history'][m]['clearances_blocks_interceptions'])
                recoveries = int(fpl_data['elements'][i]['history']['history'][m]['recoveries'])
                key_passes = int(fpl_data['elements'][i]['history']['history'][m]['key_passes'])
                tackles = int(fpl_data['elements'][i]['history']['history'][m]['tackles'])
                attempted_passes = int(fpl_data['elements'][i]['history']['history'][m]['attempted_passes'])
                completed_passes = int(fpl_data['elements'][i]['history']['history'][m]['completed_passes'])
                big_chances_missed = int(fpl_data['elements'][i]['history']['history'][m]['big_chances_missed'])
                errors_leading_to_goal = int(fpl_data['elements'][i]['history']['history'][m]['errors_leading_to_goal'])
                errors_leading_to_goal_attempt = int(fpl_data['elements'][i]['history']['history'][m]['errors_leading_to_goal_attempt'])
                tackled = int(fpl_data['elements'][i]['history']['history'][m]['tackled'])
                penalties_saved = int(fpl_data['elements'][i]['history']['history'][m]['penalties_saved'])
                saves = int(fpl_data['elements'][i]['history']['history'][m]['saves'])
                penalties_conceded = int(fpl_data['elements'][i]['history']['history'][m]['penalties_conceded'])

                total_points = int(fpl_data['elements'][i]['history']['history'][m]['total_points'])
                
                series = pd.Series([element,GW,minutes,clean_sheets,goals_conceded,own_goals,penalties_missed,
                    yellow_cards,red_cards,bonus,bps,influence,creativity,threat,ict_index,
                    big_chances_created,clearances_blocks_interception,recoveries,key_passes,tackles,
                    attempted_passes,completed_passes,big_chances_missed,errors_leading_to_goal,errors_leading_to_goal_attempt,
                    tackled,penalties_saved,saves,penalties_conceded,total_points],index=column_names)
            #threshold = threshold_points
            #if total_points >= threshold_points:
            #    passed = 1
            #else:
            #    passed = 0

            df = df.append(series,ignore_index=True)

print(" ")
print("Players full history DataFrame successfully filled!")

ID:  21  Name:  Welbeck
ID:  22  Name:  Lacazette
ID:  23  Name:  Aubameyang
ID:  552  Name:  Nketiah
ID:  43  Name:  Wilson
ID:  44  Name:  Mousset
ID:  45  Name:  King
ID:  46  Name:  Defoe
ID:  62  Name:  Murray
ID:  63  Name:  Hemed
ID:  64  Name:  Baldock
ID:  65  Name:  Locadia
ID:  66  Name:  Andone
ID:  551  Name:  Gyokeres
ID:  84  Name:  Vokes
ID:  85  Name:  Barnes
ID:  86  Name:  Wells
ID:  87  Name:  Wood
ID:  495  Name:  Vydra
ID:  107  Name:  Zohore
ID:  108  Name:  Bogle
ID:  109  Name:  Madine
ID:  110  Name:  Reid
ID:  133  Name:  Giroud
ID:  134  Name:  Morata
ID:  496  Name:  Abraham
ID:  151  Name:  Zaha
ID:  152  Name:  Benteke
ID:  153  Name:  Sørloth
ID:  444  Name:  Wickham
ID:  505  Name:  Ayew
ID:  175  Name:  Calvert-Lewin
ID:  176  Name:  Niasse
ID:  177  Name:  Tosun
ID:  470  Name:  Sandro
ID:  189  Name:  Kamara
ID:  480  Name:  Mitrovic
ID:  513  Name:  Vietto
ID:  209  Name:  Quaner
ID:  210  Name:  Depoitre
ID:  211  Name:  Mounie
ID:  212  Name:  Kac

In [82]:
#Cleaning the Dataframe
col_names_float = ['influence','creativity','threat','ict_index']   #Names of features that require to be of type float
df = df_col_astype_int(df,column_names,col_names_float)  #Changes features in df to correct type (int and floats only) 
df.head()


        #...... continue from here ............

,element,GW,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_missed,yellow_cards,...,completed_passes,big_chances_missed,errors_leading_to_goal,errors_leading_to_goal_attempt,tackled,offside,target_missed,fouls,dribbles,total_points
0,21,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,21,3,15,1,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,8
3,21,4,6,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,1
4,21,5,10,0,0,0,1,0,0,0,...,3,0,0,0,1,0,1,1,0,1
